<a href="https://colab.research.google.com/github/JiSu0708/ESAA-YB/blob/main/0519_%EC%97%B0%EC%8A%B5%EB%AC%B8%EC%A0%9C_%ED%9A%8C%EA%B7%801_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. 단순 선형 회귀 분석**
- 전복의 나이를 예측하는 선형회귀모델을 생성하세요.
- 전복의 ‘성별’, ‘키’, ‘지름’, ‘높이’, ‘전체무게’, ‘몸통무게’, ‘내장무게’, ‘껍질무게’를 이용해 ‘껍질의 고리 수’를 예측한 뒤, **예측된 ‘껍질의 고리 수’에 1.5를 더하면 전복의 나이**가 됩니다.

In [1]:
# 기본 모듈 불러오기
import numpy as np
import pandas as pd

**1) 데이터 load 및 변형** 

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# 데이터 로드
data = pd.read_csv("/content/drive/MyDrive/23-1 ESAA YB/data/abalone.csv")
data.head()
print(data.shape)

# 성별 M은 Male, F는 Female, I는 Infant 이므로 따로 열 만들기
for label in "MFI":
    data[label] = data["Sex"] == label
data.drop('Sex', axis=1, inplace=True)

(4177, 9)


**2) X, y 선택**
: y는 Rings열, X는 Rings열을 제외한 나머지를 선택하되 전부 실수가 되도록 한다.

In [4]:
data.head()

,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Rings,M,F,I
0,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15,True,False,False
1,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7,True,False,False
2,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9,False,True,False
3,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10,True,False,False
4,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7,False,False,True


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4177 entries, 0 to 4176
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Length          4177 non-null   float64
 1   Diameter        4177 non-null   float64
 2   Height          4177 non-null   float64
 3   Whole weight    4177 non-null   float64
 4   Shucked weight  4177 non-null   float64
 5   Viscera weight  4177 non-null   float64
 6   Shell weight    4177 non-null   float64
 7   Rings           4177 non-null   int64  
 8   M               4177 non-null   bool   
 9   F               4177 non-null   bool   
 10  I               4177 non-null   bool   
dtypes: bool(3), float64(7), int64(1)
memory usage: 273.4 KB


In [6]:
# X,y 데이터 선택
y = data.Rings.values
data.drop('Rings', axis=1, inplace=True)

X = data.values.astype(float)


 **3) train/test set 분리**

In [7]:
# 필요한 모듈 불러오기
from sklearn.model_selection import train_test_split

In [8]:
# train과 test set 분리 (train:test = 7:3 비율로)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 100)

**4) 선형회귀모델 생성, 모델 예측치 구하기**

In [9]:
#필요한 모듈 불러오기
from sklearn.linear_model import LinearRegression

In [10]:
#선형회귀모델 생성 및 훈련
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [11]:
# 모델 예측치 구하기
y_pred = model.predict(X_test)

# 모델 예측치를 활용해 최종적으로 전복의 나이를 예측
y_pred = y_pred + 1.5  

**5) 모델 평가: MSE, RMSE, R2 score, corr 구하기**

In [12]:
#필요한 모듈 불러오기
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

- MSE, RMSE 

In [15]:
#mse, rmse
mse = mean_squared_error(y_test, y_pred)
rmse=np.sqrt(mse)

print('MSE:{0:.3f}, RMSE:{1:.3f}'.format(mse, rmse))

MSE:7.140, RMSE:2.672


- R2 score

In [16]:
#R2 score 측정
print('R2 score: {0:.3f}'.format(r2_score(y_test, y_pred)))

R2 score: 0.325


- 회귀 절편값

In [17]:
#회귀 절편 값
model.intercept_

48785761923294.74

- 회귀 계수 값

In [19]:
#회귀 계수 값
model.coef_

array([ 1.63413000e+00,  9.22691217e+00,  7.14174151e+00,  1.00267726e+01,
       -2.14743502e+01, -1.04465071e+01,  7.85007782e+00, -4.87857619e+13,
       -4.87857619e+13, -4.87857619e+13])

- 상관계수

Hint: corr 함수 이용.

In [26]:
# 상관계수 구하기
df = pd.DataFrame({'y_pred': y_pred, 'y_true': y_test})

cor = df['y_pred'].corr(df['y_true'])
cor

0.7305149986514939

# **2. Polynomial features**

In [27]:
# PolynomialFeatures 라이브러리 호출
from sklearn.preprocessing import PolynomialFeatures

In [28]:
# 임의 데이터 생성

X = np.arange(6).reshape(3, 2)

df =  pd.DataFrame(X)
df.columns = ['x_1','x_2']
df

,x_1,x_2
0,0,1
1,2,3
2,4,5


In [30]:
# 차원은 2로 설정
poly_features = PolynomialFeatures(degree = 2)

# fit_transform 메소드를 통해 데이터 변환
df_poly = poly_features.fit_transform(df)

# PolynomialFeatures로 변환 된 데이터를 데이터 프레임 형태로 변환
df_poly = pd.DataFrame(df_poly)
df_poly

,0,1,2,3,4,5
0,1.0,0.0,1.0,0.0,0.0,1.0
1,1.0,2.0,3.0,4.0,6.0,9.0
2,1.0,4.0,5.0,16.0,20.0,25.0


In [31]:
# df_poly의 컬럼을 1,x1,x2,x1^2,x1*x2,x2^2 로 변경
df_poly.columns  = ['1','x1','x2','x1^2','x1*x2','x2^2']
df_poly

,1,x1,x2,x1^2,x1*x2,x2^2
0,1.0,0.0,1.0,0.0,0.0,1.0
1,1.0,2.0,3.0,4.0,6.0,9.0
2,1.0,4.0,5.0,16.0,20.0,25.0
